In [5]:
#below imports everthing we need
import pandas as pd
import numpy as np
import requests
import warnings
import sys
import json


mlbsong=pd.read_csv('MLB_Walkup.csv')
mlbplayer=input("Enter an active MLB player or type 'quit': ")

songdata = mlbsong
songdf = pd.DataFrame(songdata)

while True:
    try:
        #if user enters 'quit', end program
        if mlbplayer == 'quit':
            break        
        
        #makes dataframe for team, player, artist, song, playlist link
        row =(songdf.loc[songdf['Player'] == mlbplayer])
        row = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist']})
        #row

        #makes playlist link clickable
        def make_clickable(val):
            return '<a target="_blank" href="{}">{}</a>'.format(val, val)

        row.style.format({'Playlist': make_clickable})

        if songdf['Player'].count == 0:
            print("player doesn't exist")
        else:

            #beginning of mlb api
            url=("http://lookup-service-prod.mlb.com//json/named.search_player_all.bam?sport_code='mlb'&active_sw='Y'&name_part='%s'" % (mlbplayer)) 
            response=requests.get(url)
            response.ok

            #below gets player id for url2 and url3 as well as position to determine whether player is batter or pitcher
            player=response.json()
            playerid=player['search_player_all']['queryResults']['row']['player_id']
            position=player['search_player_all']['queryResults']['row']['position']
            playername=player['search_player_all']['queryResults']['row']['name_display_first_last']

            #creates picture from mlb.com's HTML
            from IPython.display import display, HTML
            def picture():
                display (HTML("<img src='https://content.mlb.com/images/headshots/current/60x60/%s@3x.png'>"%(playerid)))

            if position == "P":
                #below is for pitchers
                purl=("http://lookup-service-prod.mlb.com//json/named.sport_pitching_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response3=requests.get(purl)
                response3.ok
                p_stats=response3.json()
                wins=p_stats['sport_pitching_tm']['queryResults']['row']['w']
                innings=p_stats['sport_pitching_tm']['queryResults']['row']['ip']
                strikeouts=p_stats['sport_pitching_tm']['queryResults']['row']['so']
                pitchers = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist'],'Wins': wins, 'Innings': innings, 'Strikeouts': strikeouts})
                picture()
                print(pitchers)
                break 
            
            else:
                #below is for batters
                burl=("http://lookup-service-prod.mlb.com//json/named.sport_hitting_tm.bam?league_list_id='mlb'&game_type='R'&season='2018'&player_id='%s'" % (playerid))
                response2=requests.get(burl)
                response2.ok
                b_stats=response2.json()
                average=b_stats['sport_hitting_tm']['queryResults']['row']['avg']
                hr=b_stats['sport_hitting_tm']['queryResults']['row']['hr']
                rbi=b_stats['sport_hitting_tm']['queryResults']['row']['rbi']
                batters = pd.DataFrame({'Team': row['Team'],'Player': row['Player'], 'Artist': row['Artist'], 'Song': row['Song'], 'Playlist': row['Playlist'], 'Average': average, 'Home Runs': hr, 'RBI': rbi})
                picture()
                print(batters)
                break      
            
    except KeyError:
        print("This player does not exist, is spelled wrong, was not active in 2018, or is not in the database of mlb.com/entertainment")
        break


Enter an active MLB player or type 'quit': Clayton Kershaw


        Team           Player            Artist                     Song  \
169  Dodgers  Clayton Kershaw  The Chainsmokers  Let You Go - Radio Edit   

                  Playlist Wins Innings Strikeouts  
169  https://goo.gl/9zieev    9   161.1        155  


In [6]:
#we need to make batter dataframe (bdf) and pitcher dataframe (pdf) everthing else below is how we are going to merge it, I think

bmerged_dataframe = songdf.merge(batters, left_on='Player', right_on='Player', how='inner')
bmerged_dataframe.head()

pmerged_dataframe = songdf.merge(pitchers, left_on='Player', right_on='Player', how='inner')
pmerged_dataframe.head()

,Team_x,Player,Artist_x,Song_x,Playlist_x,Unnamed: 5,Unnamed: 6,Team_y,Artist_y,Song_y,Playlist_y,Wins,Innings,Strikeouts
0,Dodgers,Clayton Kershaw,The Chainsmokers,Let You Go - Radio Edit,https://goo.gl/9zieev,NaN,NaN,Dodgers,The Chainsmokers,Let You Go - Radio Edit,https://goo.gl/9zieev,9,161.1,155


In [ ]:
bdg